# UCB Extension Data Science Homework 4
Academy Exercise 
written by: A. Lam

# Analysis
- Observation 1:
- Observation 2:
- Observation 3:

In [1]:
# Modules
import os
import pandas as pd
import numpy as np

In [2]:
# Create path to File
# file 1
school_path = os.path.join('..','Instructions','PyCitySchools','raw_data','schools_complete.csv')
# file 2
student_path = os.path.join('..','Instructions','PyCitySchools','raw_data','students_complete.csv')

In [3]:
# Read file
school_data = pd.read_csv(school_path)
student_data = pd.read_csv(student_path)

# District Summary

Create overall district metrics incl. % passing in math and reading, total students, etc.
Will need to bin students into passing or failing on scores, and subsequently group over school.

In [4]:
# Categorize students as passing or failing
bins = [0,69,100]
score = ['failing','passing']
student_data['reading_result'] = pd.cut(student_data['reading_score'],bins,labels = score)
student_data['math_result'] = pd.cut(student_data['math_score'],bins,labels = score)
# student_data[['math_result','math_score','reading_result','reading_score']].head()

In [5]:
# calculate values
n_schools = school_data['name'].nunique()
n_students = student_data['name'].count()
tot_budget = school_data['budget'].sum()
mean_math = student_data['math_score'].mean()
mean_reading = student_data['reading_score'].mean()
pcnt_pass_math = 100*student_data['math_result'].value_counts()['passing'] / n_students
pcnt_pass_reading = 100*student_data['reading_result'].value_counts()['passing'] / n_students
oall_pass = 100*student_data[(student_data['math_result'] == 'passing') & 
              (student_data['reading_result'] == 'passing')].count()['Student ID'] / n_students

In [6]:
# create dataframe
district_summary = pd.DataFrame({
    'Total Schools' : n_schools,
    'Total Students' : n_students,
    'Total Budget' : tot_budget,
    'Average Math Score' : mean_math,
    'Average Reading Score' : mean_reading,
    '% Passing Math' : pcnt_pass_math,
    '% Passing Reading' : pcnt_pass_reading,
    'Overall Passing Rate' : oall_pass},index =['District'])
district_summary.head()

,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Passing Rate,Total Budget,Total Schools,Total Students
District,74.980853,85.805463,78.985371,81.87784,65.172326,24649428,15,39170


# School Summary

In [7]:
school_data.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [8]:
student_data.head()

,Student ID,name,gender,grade,school,reading_score,math_score,reading_result,math_result
0,0,Paul Bradley,M,9th,Huang High School,66,79,failing,passing
1,1,Victor Smith,M,12th,Huang High School,94,61,passing,failing
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,passing,failing
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,failing,failing
4,4,Bonnie Ray,F,9th,Huang High School,97,84,passing,passing


In [9]:
# replace 'passing' with True and 'failing' with False
def res2bool(arg):
    if arg == 'passing':
        return True
    elif arg == 'failing':
        return False
    else:
        return float('nan')
student_data_bool = student_data[['reading_result','math_result']].applymap(res2bool)
student_data_bool.columns = ['r_bool','m_bool']
# use logical & to see students that passed both math and reading
student_data_bool['oall_bool'] = student_data_bool['r_bool']&student_data_bool['m_bool']
student_data_bool.head()

,r_bool,m_bool,oall_bool
0,False,True,False
1,True,False,False
2,True,False,False
3,False,False,False
4,True,True,True


In [10]:
# rejoin booleans to student_data to compute stats after groupby
student_data = pd.merge(student_data,student_data_bool,left_index=True,right_index=True)
students_by_school = student_data.groupby('school')
student_performance = students_by_school[['math_score','reading_score']].mean()
student_performance.columns = ['Average Math Score','Average Reading Score']

In [11]:
passing_rates_by_school = 100*students_by_school[['r_bool','m_bool','oall_bool']].sum()/students_by_school[['r_bool','m_bool','oall_bool']].count()
passing_rates_by_school.columns=['% Passing Reading', '% Passing Math', 'Overall Passing Rate']
passing_rates_by_school.head()

,% Passing Reading,% Passing Math,Overall Passing Rate
school,,,
Bailey High School,81.933280,66.680064,54.642283
Cabrera High School,97.039828,94.133477,91.334769
Figueroa High School,80.739234,65.988471,53.204476
Ford High School,79.299014,68.309602,54.289887
Griffin High School,97.138965,93.392371,90.599455


In [12]:
# combine dataframes
school_summary = pd.merge(school_data,student_performance,left_on='name',right_index=True)
school_summary = pd.merge(school_summary,passing_rates_by_school,left_on='name',right_index=True)
school_summary.rename(columns = {
    'size':'Total Students',
    'budget':'Total Budget',
    'name':'School Name',
    'type':'School Type'
    },inplace=True)
school_summary['Budget Per Student'] = school_summary['Total Budget']/school_summary['Total Students']
school_summary.head()

,School ID,School Name,School Type,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,Overall Passing Rate,Budget Per Student
0,0,Huang High School,District,2917,1910635,76.629414,81.182722,81.316421,65.683922,53.513884,655.0
1,1,Figueroa High School,District,2949,1884411,76.711767,81.158020,80.739234,65.988471,53.204476,639.0
2,2,Shelton High School,Charter,1761,1056600,83.359455,83.725724,95.854628,93.867121,89.892107,600.0
3,3,Hernandez High School,District,4635,3022020,77.289752,80.934412,80.862999,66.752967,53.527508,652.0
4,4,Griffin High School,Charter,1468,917500,83.351499,83.816757,97.138965,93.392371,90.599455,625.0


# Top Performing School (By Passing Rate)

In [13]:
school_summary_sorted = school_summary.sort_values(by='Overall Passing Rate',ascending=False)
school_summary_sorted.head()

,School ID,School Name,School Type,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,Overall Passing Rate,Budget Per Student
6,6,Cabrera High School,Charter,1858,1081356,83.061895,83.975780,97.039828,94.133477,91.334769,582.0
14,14,Thomas High School,Charter,1635,1043130,83.418349,83.848930,97.308869,93.272171,90.948012,638.0
4,4,Griffin High School,Charter,1468,917500,83.351499,83.816757,97.138965,93.392371,90.599455,625.0
5,5,Wilson High School,Charter,2283,1319574,83.274201,83.989488,96.539641,93.867718,90.582567,578.0
9,9,Pena High School,Charter,962,585858,83.839917,84.044699,95.945946,94.594595,90.540541,609.0


# Math Scores by Grade

In [14]:
students_by_school_and_grade = student_data.groupby(['school','grade'])
math_by_grade = students_by_school_and_grade['math_score'].mean()

In [27]:
students_by_school_and_grade[school_names[0],'10th']

KeyError: "Columns not found: '10th', 'Huang High School'"

# Reading Scores by Grade

In [16]:
students_by_school_and_grade['reading_score'].mean().head(12)

school                grade
Bailey High School    10th     80.907183
                      11th     80.945643
                      12th     80.912451
                      9th      81.303155
Cabrera High School   10th     84.253219
                      11th     83.788382
                      12th     84.287958
                      9th      83.676136
Figueroa High School  10th     81.408912
                      11th     80.640339
                      12th     81.384863
                      9th      81.198598
Name: reading_score, dtype: float64